In [2]:
import sys
import os
import pandas as pd
current_file_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_file_path, '..'))  # ,'..','..'
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)


from load_inputs.weather import load_data as load_data_subway_in,START, END,FILE_NAME
from plotting.TS_analysis import plot_TS
from argparse import Namespace
from examples.benchmark import local_get_args
from constants.paths import FOLDER_PATH

invalid_dates = []
minmaxnorm = True
standardize = False  # Set to True if you want to standardize the data
config = {'model_name': 'STGCN',
         'freq' : '15min',
        'dataset_names':['weather'],
        'dataset_for_coverage' : ['subway_in'],
        'target_data': 'weather',
        'contextual_kwargs':{'weather':{},
                           },
        'target_kwargs':{'weather':{'quantile_filter_outliers':0.995,}},
        'step_ahead':1,
        'horizon_step':1,
        }

args = local_get_args(config['model_name'],
                args_init = None,
                dataset_names=config['dataset_names'],
                dataset_for_coverage=config['dataset_for_coverage'],
                modification = config)



coverage_period = pd.date_range(start=START, end=END, freq='15min')[:-1]  # Exclude the last date to match the data
args_out = Namespace(**vars(args))  
args_out.target_data = 'weather'
args_out.dataset_for_coverage = ['weather']
args_out.dataset_names = ['weather']
args_out.contextual_kwargs = {}


ds = load_data_subway_in(FOLDER_PATH, coverage_period=coverage_period, invalid_dates=invalid_dates, args=args_out, minmaxnorm=minmaxnorm,standardize=standardize, normalize=True,filename=FILE_NAME,
              tensor_limits_keeper = None)
df =  pd.DataFrame(ds.U_train[:,:,-1].detach().cpu().numpy(),columns = ds.spatial_unit, index = ds.tensor_limits_keeper.df_verif_train.iloc[:,-2] )

plot_TS(df.iloc[:,:],bool_show = True, height = 800)

/home/rrochas/prediction-validation/load_inputs/weather.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_weather = df_weather.applymap(lambda x: float(x.replace(',','.')) if isinstance(x,str) else x)


Values with issues:  0.001%
Regular Values that we have to set to 0:  0.000%
Values with issues:  0.014%
Regular Values that we have to set to 0:  0.000%


Loading BokehJS ...

figure(id='p1074', ...)

In [10]:
pd.DataFrame(ds.raw_values).fillna(0).max(),df.max()

(0    26.5
 1    61.5
 dtype: float32,
 0    30.916758
 1    40.388847
 dtype: float32)